# Importação e instalação de bibliotecas

In [ ]:
pip install pandera

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
# import pandera as pa
import psycopg2
from cassandra.cluster import Cluster
from cassandra.connection import EndPoint

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.window import Window

# Migração Postgres (nuvem) para Python (Pandas)

In [ ]:
# Definição do conector

class connector_postgres:
    host = "34.151.204.5"
    database = "bc_reclame"
    user = "postgres"
    password = "secret"

    def conectar(self):
        conn = psycopg2.connect(host=self.host,
                                database=self.database,
                                user=self.user,
                                password=self.password)
        cursor = conn.cursor()

        return conn, cursor

    def desconectar(self, conn, cursor):
        cursor.close()
        conn.commit()
        conn.close()

    def selecionar(self, query):
        conn, cursor = self.conectar()
        cursor.execute(query)
        lista = []
        for result in cursor.fetchall():
            lista.append(result)
        self.desconectar(conn, cursor)
        return lista
            

conect = connector_postgres()


In [ ]:
# Migração através de Queries

query1 = "SELECT * FROM bc_reclame_sem"

bc_reclame_sem = pd.DataFrame(conect.selecionar(query1), columns=['ano', 'semestre', 'categoria','tipo','instituicao_financeira', 'indice', 'qtde_rec_reg_proc', 'qtde_rec_reg_nao_proc','qtde_rec_nao_reg', 'qtde_total_rec', 'qtde_total_clientes'])


query2 = "SELECT * FROM bc_reclame_tri"

bc_reclame_tri = pd.DataFrame(conect.selecionar(query2), columns=['ano', 'trimestre', 'categoria','tipo','instituicao_financeira', 'indice', 'qtde_rec_reg_proc', 'qtde_rec_reg_nao_proc','qtde_rec_nao_reg', 'qtde_total_rec', 'qtde_total_clientes'])


In [ ]:
# Conferindo o sucesso da operação

bc_reclame_sem

In [ ]:
# Conferindo o sucesso da operação

bc_reclame_tri

# Transformações Pandas

## Preparação para concatenar

In [ ]:
# Conferindo os tipos de dados

bc_reclame_sem.dtypes

In [ ]:
# Renomear columa semestre para periodo

bc_reclame_sem.rename(columns={'semestre': 'periodo'}, inplace = True)

In [ ]:
# Alterando valor na coluna periodo, de 1S e 2S para 1 semestre e 2 semestre

bc_reclame_sem.loc[bc_reclame_sem['periodo'] == '1∫', ['periodo']] = '1 semestre'
bc_reclame_sem.loc[bc_reclame_sem['periodo'] == '2∫', ['periodo']] = '2 semestre'

In [ ]:
# Conferindo o sucesso da operação

bc_reclame_sem.loc[bc_reclame_sem['periodo'] == '1 semestre']

In [ ]:
# Renomear columa trimestre para periodo

bc_reclame_tri.rename(columns={'trimestre': 'periodo'}, inplace = True)

In [ ]:
# Alterando valor na coluna periodo

bc_reclame_tri.loc[bc_reclame_tri['periodo'] == '1∫', ['periodo']] = '1 trimestre'
bc_reclame_tri.loc[bc_reclame_tri['periodo'] == '2∫', ['periodo']] = '2 trimestre'
bc_reclame_tri.loc[bc_reclame_tri['periodo'] == '3∫', ['periodo']] = '3 trimestre'
bc_reclame_tri.loc[bc_reclame_tri['periodo'] == '4∫', ['periodo']] = '4 trimestre'

In [ ]:
# Verificando o sucesso da operação

bc_reclame_tri

## Concatenar os DFs semestre e trimestre

In [ ]:
# Concatenar os dois DFs

bc_reclame = pd.concat([bc_reclame_sem, bc_reclame_tri], axis = 0, ignore_index = True)

In [ ]:
# Verificando o sucesso da operação

bc_reclame

In [ ]:
# Fazer backup do DF

bc_reclame_backup = bc_reclame.copy()

## Limpeza e normalização

In [ ]:
bc_reclame.dtypes

In [ ]:
bc_reclame.head(1)

In [ ]:
# Verificando inconsistências na coluna 'ano'

sorted(pd.unique(bc_reclame['ano']))

In [ ]:
# Alterando o tipo de dado na coluna 'ano'

bc_reclame = bc_reclame.astype({'ano': int})

In [ ]:
bc_reclame.dtypes

In [ ]:
# Verificando inconsistências na coluna 'periodo'

sorted(pd.unique(bc_reclame['periodo']))

In [ ]:
# Verificando inconsistências na coluna 'categoria'

sorted(pd.unique(bc_reclame['categoria']))

In [ ]:
# Excluindo caracteres estranhos da coluna 'categoria'

bc_reclame.loc[bc_reclame['categoria'] == 'Mais de dois milhıes de clientes', ['categoria']] = 'Mais de dois milhoes de clientes'
bc_reclame.loc[bc_reclame['categoria'] == 'Menos de dois milhıes de clientes', ['categoria']] = 'Menos de dois milhoes de clientes'
bc_reclame.loc[bc_reclame['categoria'] == 'Grupo Secund·rio', ['categoria']] = 'Grupo Secundario'

In [ ]:
# Verificando o resultado da operação

sorted(pd.unique(bc_reclame['categoria']))

In [ ]:
# Verificando inconsistências na coluna 'tipo'

sorted(pd.unique(bc_reclame['tipo']))

In [ ]:
# Verificando inconsistências na coluna 'instituicao_financeira'

sorted(pd.unique(bc_reclame['instituicao_financeira']))

In [ ]:
# Verificando instituições financeiras que tem nomes muito próximos - possivelmente são as mesmas

# 'BANCO BM FBOVESPA DE SERVICOS DE LIQUIDACAO E CUSTODIA S A'
# 'BANCO BM&FBOVESPA DE SERVI«OS DE LIQUIDA«√O E CUST”DIA S.A.'

bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BM FBOVESPA DE SERVICOS DE LIQUIDACAO E CUSTODIA S A']

In [ ]:
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BM&FBOVESPA DE SERVI«OS DE LIQUIDA«√O E CUST”DIA S.A.']

In [ ]:
# Verificando o número de instituições financeiras

len(pd.unique(bc_reclame['instituicao_financeira']))

In [ ]:
# Alterando valores na coluna instituicao_financeira, de modo a eliminar dois tipos de inconsistências
# 1) em dados que deveriam ser iguais, mas foram escritos de maneira ligeiramente distinta
# 2) dados com caracteres estranhos

bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'AGIPLAN FINANCEIRA S A    CREDITO FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'AGIPLAN FINANCEIRA S.A CREDITO FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'AGORACRED S A SOCIEDADE DE CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'AGORACRED S.A. SOCIEDADE DE CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'AGORACRED S/A SOCIEDADE DE CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'AGORACRED S.A. SOCIEDADE DE CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'AL5 S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'AL5 S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'AMAGGI S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'AMAGGI S.A CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'AVISTA S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'AVISTA S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO A J  RENNER S A', ['instituicao_financeira']] = 'BANCO A.J. RENNER S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO ABN AMRO S A', ['instituicao_financeira']] = 'BANCO ABN AMRO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO ARBI S A', ['instituicao_financeira']] = 'BANCO ARBI S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO AZTECA DO BRASIL S A', ['instituicao_financeira']] = 'BANCO AZTECA DO BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BARCLAYS S A', ['instituicao_financeira']] = 'BANCO BARCLAYS S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BM FBOVESPA DE SERVICOS DE LIQUIDACAO E CUSTODIA S A', ['instituicao_financeira']] = 'BANCO BM FBOVESPA DE SERVICOS DE LIQUIDACAO E CUSTODIA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BM&FBOVESPA DE SERVI«OS DE LIQUIDA«√O E CUST”DIA S.A', ['instituicao_financeira']] = 'BANCO BM FBOVESPA DE SERVICOS DE LIQUIDACAO E CUSTODIA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BONSUCESSO S A', ['instituicao_financeira']] = 'BANCO BONSUCESSO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BRACCE S A', ['instituicao_financeira']] = 'BANCO BRACCE S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BRJ S A', ['instituicao_financeira']] = 'BANCO BRJ S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO CAPITAL S A', ['instituicao_financeira']] = 'BANCO CAPITAL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO CATERPILLAR S A', ['instituicao_financeira']] = 'BANCO CATERPILLAR S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO CEDULA S A', ['instituicao_financeira']] = 'BANCO CEDULA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO CNH INDUSTRIAL CAPITAL S A', ['instituicao_financeira']] = 'BANCO CNH INDUSTRIAL CAPITAL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO CSF S A', ['instituicao_financeira']] = 'BANCO CSF S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DA AMAZONIA S A', ['instituicao_financeira']] = 'BANCO DA AMAZONIA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DA CHINA BRASIL S A', ['instituicao_financeira']] = 'BANCO DA CHINA BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DAYCOVAL S A', ['instituicao_financeira']] = 'BANCO DAYCOVAL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DE LAGE LANDEN BRASIL S A', ['instituicao_financeira']] = 'BANCO DE LAGE LANDEN BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DE TOKYO MITSUBISHI UFJ BRASIL S A', ['instituicao_financeira']] = 'BANCO DE TOKYO-MITSUBISHI UFJ BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DO ESTADO DE SERGIPE S A', ['instituicao_financeira']] = 'BANCO DO ESTADO DE SERGIPE S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DO ESTADO DO PARA S A', ['instituicao_financeira']] = 'BANCO DO ESTADO DO PARA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DO ESTADO DO PAR¡ S.A.', ['instituicao_financeira']] = 'BANCO DO ESTADO DO PARA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO DO NORDESTE DO BRASIL S A', ['instituicao_financeira']] = 'BANCO DO NORDESTE DO BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO FIBRA S A', ['instituicao_financeira']] = 'BANCO FIBRA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO FICSA S A', ['instituicao_financeira']] = 'BANCO FICSA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO FIDIS S A', ['instituicao_financeira']] = 'BANCO FIDIS S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO FIDIS S/A', ['instituicao_financeira']] = 'BANCO FIDIS S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO FORD S A', ['instituicao_financeira']] = 'BANCO FORD S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO GERADOR S A', ['instituicao_financeira']] = 'BANCO GERADOR S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO IBM S A', ['instituicao_financeira']] = 'BANCO IBM S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO INBURSA DE INVESTIMENTOS S A', ['instituicao_financeira']] = 'BANCO INBURSA DE INVESTIMENTOS S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO INDUSCRED DE INVESTIMENTO S A', ['instituicao_financeira']] = 'BANCO INDUSCRED DE INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO KDB DO BRASIL S A', ['instituicao_financeira']] = 'BANCO KDB DO BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO KEB DO BRASIL S A', ['instituicao_financeira']] = 'BANCO KEB DO BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO KEB HANA DO BRASIL S A', ['instituicao_financeira']] = 'BANCO KEB HANA DO BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO LUSO BRASILEIRO S A', ['instituicao_financeira']] = 'BANCO LUSO BRASILEIRO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO MODAL S A', ['instituicao_financeira']] = 'BANCO MODAL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO MONEO S A', ['instituicao_financeira']] = 'BANCO MONEO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO POTTENCIAL S A', ['instituicao_financeira']] = 'BANCO POTTENCIAL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO RABOBANK INTERNATIONAL BRASIL S A', ['instituicao_financeira']] = 'BANCO RABOBANK INTERNATIONAL BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO POTTENCIAL S A', ['instituicao_financeira']] = 'BANCO POTTENCIAL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO RABOBANK INTERNATIONAL BRASIL S A', ['instituicao_financeira']] = 'BANCO RABOBANK INTERNATIONAL BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO SEMEAR S A', ['instituicao_financeira']] = 'BANCO SEMEAR S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO STANDARD DE INVESTIMENTOS S A', ['instituicao_financeira']] = 'BANCO STANDARD DE INVESTIMENTOS S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO TOPAZIO S A', ['instituicao_financeira']] = 'BANCO TOPAZIO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO TOP¡ZIO S.A.', ['instituicao_financeira']] = 'BANCO TOPAZIO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO TOYOTA DO BRASIL S A', ['instituicao_financeira']] = 'BANCO TOYOTA DO BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO TRIANGULO S A', ['instituicao_financeira']] = 'BANCO TRIANGULO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO TRICURY S A', ['instituicao_financeira']] = 'BANCO TRICURY S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO VIPAL S A', ['instituicao_financeira']] = 'BANCO VIPAL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO WOORI BANK DO BRASIL S A', ['instituicao_financeira']] = 'BANCO WOORI BANK DO BRASIL S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANIF - BANCO INTERNACIONAL DO FUNCHAL (BRASIL), S.A. - EM LIQUIDA«√O ORDIN¡RIA', ['instituicao_financeira']] = 'BANIF - BANCO INTERNACIONAL DO FUNCHAL (BRASIL), S.A. - EM LIQUIDACAOO ORDINARIA'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BECKER FINANCEIRA S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'BECKER FINANCEIRA S.A. - CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BIORC FINANCEIRA   CREDITO  FINANCIAMENTO E INVESTIMENTO S A', ['instituicao_financeira']] = 'BIORC FINANCEIRA   CREDITO  FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BIORC FINANCEIRA - CR…DITO, FINANCIAMENTO E INVESTIMENTO S.A', ['instituicao_financeira']] = 'BIORC FINANCEIRA - CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BPN BRASIL BANCO MULTIPLO S A', ['instituicao_financeira']] = 'BPN BRASIL BANCO MULTIPLO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BRICKELL S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'BRICKELL S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BRK S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'BRK S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'CAIXA ECON‘MICA FEDERAL (conglomerado)', ['instituicao_financeira']] = 'CAIXA ECONOMICA FEDERAL (conglomerado)'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'CARUANA S A    SOCIEDADE DE CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'CARUANA S.A. SOCIEDADE DE CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'CARUANA S.A. - SOCIEDADE DE CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'CARUANA S.A. SOCIEDADE DE CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'CENTROCRED S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'CENTROCRED S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'CREDIARE S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'CREDIARE S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'CREDITA S A CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'CREDITA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'CREFISA S A  CREDITO  FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'CREFISA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'CREFISA S.A.-CREDITO, FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'CREFISA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'DREBES FINANCEIRA S A  CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'DREBES FINANCEIRA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'FACTA FINANCEIRA S A    CREDITO FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'FACTA FINANCEIRA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'FACTA FINANCEIRA S.A. - CR…DITO FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'FACTA FINANCEIRA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'FC FINANCEIRA S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'FC FINANCEIRA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'FINAMAX S A    CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'FINAMAX S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'FINAMAX S.A. - CREDITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'FINAMAX S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'GRAZZIOTIN FINANCIADORA S/A - CR…DITO, FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'GRAZZIOTIN FINANCIADORA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'HS FINANCEIRA S A CREDITO  FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'HS FINANCEIRA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'INTESA SANPAOLO BRASIL S A    BANCO MULTIPLO', ['instituicao_financeira']] = 'INTESA SANPAOLO BRASIL S.A. BANCO MULTIPLO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'KREDILIG S A    CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'KREDILIG S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'KREDILIG S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'KREDILIG S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'LEBES FINANCEIRA S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'LEBES FINANCEIRA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'M PAGAMENTOS S.A. - CR…DITO FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'M PAGAMENTOS S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'MERCADO CR…DITO (conglomerado)', ['instituicao_financeira']] = 'MERCADO CREDITO (conglomerado)'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'MIDWAY S A    CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'MIDWAY S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'MIDWAY S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'MIDWAY S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'MULTIPLA CREDITO  FINANCIAMENTO E INVESTIMENTO S A', ['instituicao_financeira']] = 'MULTIPLA CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'M¡XIMA (conglomerado)', ['instituicao_financeira']] = 'MAXIMA (conglomerado)' 
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'NATIXIS BRASIL S.A. BANCO M⁄LTIPLO', ['instituicao_financeira']] = 'NATIXIS BRASIL S.A. BANCO MULTIPLO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'NEGRESCO S A    CREDITO  FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'NEGRESCO S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'NEGRESCO S.A. - CREDITO, FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'NEGRESCO S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'NOVO BANCO CONTINENTAL S A    BANCO MULTIPLO', ['instituicao_financeira']] = 'NOVO BANCO CONTINENTAL S.A. BANCO MULTIPLO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'NOVO BANCO CONTINENTAL S.A. - BANCO M⁄LTIPLO', ['instituicao_financeira']] = 'NOVO BANCO CONTINENTAL S.A. BANCO MULTIPLO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'PARAN¡ BANCO (conglomerado)', ['instituicao_financeira']] = 'PARANA BANCO (conglomerado)'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'PARATI   CREDITO  FINANCIAMENTO E INVESTIMENTO S A', ['instituicao_financeira']] = 'PARATI CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'PARATI - CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.', ['instituicao_financeira']] = 'PARATI CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'PEFISA S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'PEFISA S.A. - CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'PERNAMBUCANAS FINANCIADORA S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'PERNAMBUCANAS FINANCIADORA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'PORTOCRED S A    CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'PORTOCRED S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'PORTOCRED S.A. - CREDITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'PORTOCRED S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'REALIZE CR…DITO, FINANCIAMENTO E INVESTIMENTO S.A.', ['instituicao_financeira']] =  'REALIZE CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SANTANA S A    CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'SANTANA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SANTANA S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'SANTANA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SAX S A    CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'SAX S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SAX S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'SAX S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SCOTIABANK BRASIL S.A. BANCO M⁄LTIPLO', ['instituicao_financeira']] = 'SCOTIABANK BRASIL S.A. BANCO MULTIPLO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SENFF S A    CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'SENFF S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SENFF S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'SENFF S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO',
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SOCINAL S A    CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'SOCINAL S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SOCINAL S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'SOCINAL S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SOROCRED   CREDITO  FINANCIAMENTO E INVESTIMENTO S A', ['instituicao_financeira']] = 'SOROCRED CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'SOROCRED   CR…DITO, FINANCIAMENTO E INVESTIMENTO S.A.', ['instituicao_financeira']] = 'SOROCRED CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'TODESCREDI S A   CREDITO  FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'TODESCREDI S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'TODESCREDI S/A - CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'TODESCREDI S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'VIA CERTA FINANCIADORA S A    CREDITO  FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'VIA CERTA FINANCIADORA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'VIA CERTA FINANCIADORA S.A. - CR…DITO, FINANCIAMENTO E INVESTIMENTOS', ['instituicao_financeira']] = 'VIA CERTA FINANCIADORA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTOS'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'WILL FINANCEIRA S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'WILL FINANCEIRA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'

In [ ]:
# Verificando o número de instituições financeiras após as alterações

bc_reclame['instituicao_financeira'].nunique()

In [ ]:
# Verificando mais uma vez os valores na coluna 'instituicao_financeira'
# atentando especialmente à eliminação de nomes praticamente iguais

sorted(pd.unique(bc_reclame['instituicao_financeira']))

In [ ]:
'''
'AVISTA S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTO' - 'AVISTA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
'BANCO BM FBOVESPA DE SERVICOS DE LIQUIDACAO E CUSTODIA S.A.' - 'BANCO BM FBOVESPA DE SERVICOS DE LIQUIDACAO E CUSTODIA S.A.'
'BIORC FINANCEIRA   CREDITO  FINANCIAMENTO E INVESTIMENTO S.A.' - 'BIORC FINANCEIRA CREDITO  FINANCIAMENTO E INVESTIMENTO S.A.'
'BIORC FINANCEIRA - CR…DITO, FINANCIAMENTO E INVESTIMENTO S.A.' - 'BIORC FINANCEIRA CREDITO  FINANCIAMENTO E INVESTIMENTO S.A.'
'''

# Mais uma rodada de alterações com valores que passaram despercebidos na primeira

bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'AVISTA S.A. CR…DITO, FINANCIAMENTO E INVESTIMENTO', ['instituicao_financeira']] = 'AVISTA S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BANCO BM&FBOVESPA DE SERVI«OS DE LIQUIDA«√O E CUST”DIA S.A.', ['instituicao_financeira']] = 'BANCO BM FBOVESPA DE SERVICOS DE LIQUIDACAO E CUSTODIA S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BIORC FINANCEIRA   CREDITO  FINANCIAMENTO E INVESTIMENTO S.A.', ['instituicao_financeira']] = 'BIORC FINANCEIRA CREDITO  FINANCIAMENTO E INVESTIMENTO S.A.'
bc_reclame.loc[bc_reclame['instituicao_financeira'] == 'BIORC FINANCEIRA - CR…DITO, FINANCIAMENTO E INVESTIMENTO S.A.', ['instituicao_financeira']] = 'BIORC FINANCEIRA CREDITO  FINANCIAMENTO E INVESTIMENTO S.A.'


In [ ]:
# Verificando mais uma vez a qtde de instuiçoes financeiras

bc_reclame['instituicao_financeira'].nunique()

In [ ]:
# Verificando novamente os valores na coluna 'instituicao_financeira'

sorted(pd.unique(bc_reclame['instituicao_financeira']))

In [ ]:
# Verificando inconsistencias na coluna 'indice'

sorted(pd.unique(bc_reclame['indice']))

In [ ]:
# Renomeando coluna 'indice'

bc_reclame.rename(columns = {'indice': 'indice_rec'}, inplace = True)

In [ ]:
# Verificando o número de linhas com valores ' '

bc_reclame.loc[bc_reclame['indice_rec'] == ' ']

In [ ]:
# Função para trocar vírgula por ponto e ponto por nada na coluna 'indice_rec'

bc_reclame['indice_rec'] = bc_reclame['indice_rec'].map(lambda x: x if not "," in x else x.replace(".","").replace(",","."))

In [ ]:
# Verificando alterações

sorted(pd.unique(bc_reclame['indice_rec']))

In [ ]:
# Convertendo a coluna indice para numérico (float) 
# alterando valores não válidos para NaN

bc_reclame['indice_rec'] = pd.to_numeric(bc_reclame['indice_rec'], errors = 'coerce')

In [ ]:
# Verificando valores na coluna 'indice_rec'

sorted(pd.unique(bc_reclame['indice_rec']))

In [ ]:
bc_reclame.dtypes

In [ ]:
# Verificando inconsistências na coluna 'qtde_rec_reg_proc'

sorted(pd.unique(bc_reclame['qtde_rec_reg_proc']))

In [ ]:
# Convertendo a coluna 'qtde_rec_reg_proc' para int

bc_reclame = bc_reclame.astype({'qtde_rec_reg_proc': int})

In [ ]:
bc_reclame.dtypes

In [ ]:
# Verificando inconsistências na coluna 'qtde_rec_reg_nao_proc'

sorted(pd.unique(bc_reclame['qtde_rec_reg_nao_proc']))

In [ ]:
# Convertendo a coluna 'qtde_rec_reg_nao_proc' para int

bc_reclame = bc_reclame.astype({'qtde_rec_reg_nao_proc': int})

In [ ]:
bc_reclame.dtypes

In [ ]:
# Verificando inconsistências na coluna 'qtde_rec_nao_reg'

sorted(pd.unique(bc_reclame['qtde_rec_nao_reg']))

In [ ]:
# Convertendo a coluna 'qtde_rec_nao_reg' para int

bc_reclame = bc_reclame.astype({'qtde_rec_nao_reg': int})

In [ ]:
bc_reclame.dtypes

In [ ]:
# Verificando inconsistências na coluna 'qtde_total_rec'

sorted(pd.unique(bc_reclame['qtde_total_rec']))

In [ ]:
# Convertendo a coluna 'qtde_total_rec'

bc_reclame = bc_reclame.astype({'qtde_total_rec': int})

In [ ]:
bc_reclame.dtypes

In [ ]:
# Verificando inconsistências na coluna 'qtde_total_clientes'

sorted(bc_reclame['qtde_total_clientes'].unique())

In [ ]:
# Verificando bancos sem informações em relação aos clientes (' ')

bc_reclame.loc[bc_reclame['qtde_total_clientes'] == ' ']

In [ ]:
# Verificando instituições financeiras sem informações em relação aos clientes ('0')

bc_reclame.loc[bc_reclame['qtde_total_clientes'] == '0']

In [ ]:
# Excluindo as 15 linhas nas quais as instituições financeiras aprensentam o valor ' ' ou '0' na coluna 'qtde_total_clientes'

bc_reclame.drop(bc_reclame[(bc_reclame['qtde_total_clientes'] == ' ')].index, inplace=True)
bc_reclame.drop(bc_reclame[(bc_reclame['qtde_total_clientes'] == '0')].index, inplace=True)

In [ ]:
# Verificando se os valores foram realmente excluídos

sorted(bc_reclame['qtde_total_clientes'].unique())

In [ ]:
# Convertendo a coluna 'qtde_total_clientes'

bc_reclame = bc_reclame.astype({'qtde_total_clientes': int})

In [ ]:
bc_reclame.dtypes

# Validação Pandera

In [ ]:
#Validando Schema

schema = pa.DataFrameSchema(columns = {
        'ano': pa.Column(pa.Int),
        'periodo': pa.Column(pa.String),
        'categoria': pa.Column(pa.String),
        'tipo': pa.Column(pa.String),
        'instituicao_financeira': pa.Column(pa.String),
        'indice_rec': pa.Column(pa.Float, nullable = True),
        'qtde_rec_reg_proc': pa.Column(pa.Int),
        'qtde_rec_reg_nao_proc': pa.Column(pa.Int),
        'qtde_rec_nao_reg': pa.Column(pa.Int),
        'qtde_total_rec': pa.Column(pa.Int),
        'qtde_total_clientes': pa.Column(pa.Int)
    }
)

schema.validate(bc_reclame)

# Salvar DF normalizado em Pandas no Data Lake

In [ ]:
bc_reclame.to_csv('gs://deloitte_g10/saída/Pandas/BC-Reclame/Reclame_norm_pandas.csv', index=False)

# Migração para Cassandra

In [ ]:
# Conectando ao Cassandra

cluster = Cluster(['35.198.46.127'])
                               
session = cluster.connect('reclame_2014_2021')

In [ ]:
# Salvando os valores do DF em uma lista

lista = bc_reclame.values.tolist()

print(lista)

In [ ]:
# Passando os valores para o Cassandra

for i in lista:
    query = "INSERT INTO bc_reclame (chave, ano, periodo, categoria, tipo, instituicao_financeira, indice_rec, qtde_rec_reg_proc, qtde_rec_reg_nao_proc, qtde_rec_nao_reg, qtde_total_rec, qtde_total_clientes) VALUES(uuid(),{},'{}','{}','{}','{}','{}',{},{},{},{},{})".format(i[0], i[1], i[2], i[3], i[4], i[5], i[6], i[7], i[8], i[9], i[10])
    session.execute(query)

In [ ]:
# Desligando a conexão com o Cassandra

cluster.shutdown()

# Migração de Cassandra para PySpark

## Importando dados do Cassandra para uma lista

In [66]:
# Conectando com a VM do Cassandra
cluster = Cluster(['35.198.46.127'])
                               
session = cluster.connect('reclame_2014_2021')

In [67]:
# Importar os dados do Cassandra para uma lista

query1 = "SELECT * FROM bc_reclame"

saida = session.execute(query1)
dados = []
for i in saida:
    dados.append(i)

In [68]:
# Imprimir a lista com os dados

len(dados)

2488

In [69]:
# Cortando a conexão com o Cassandra

cluster.shutdown()

## Salvando a lista em um DF Pandas

chave, ano, categoria, indice_rec, instituicao_financeira, periodo, qtde_rec_nao_reg,
qtde_rec_reg_nao_proc, qtde_rec_reg_proc, qtde_total_clientes, qtde_total_rec

In [70]:
# Salvar os dados em um DataFrame de Pandas

bc_reclame2 = pd.DataFrame(dados)

In [71]:
# Verificando os tipos de dados

bc_reclame2.dtypes

chave                     object
ano                        int64
categoria                 object
indice_rec                object
instituicao_financeira    object
periodo                   object
qtde_rec_nao_reg           int64
qtde_rec_reg_nao_proc      int64
qtde_rec_reg_proc          int64
qtde_total_clientes        int64
qtde_total_rec             int64
tipo                      object
dtype: object

In [72]:
# Visualizando o DF

bc_reclame2

,chave,ano,categoria,indice_rec,instituicao_financeira,periodo,qtde_rec_nao_reg,qtde_rec_reg_nao_proc,qtde_rec_reg_proc,qtde_total_clientes,qtde_total_rec,tipo
0,07700578-a4b6-4d47-b79b-ff0a873c85f0,2019,Menos de quatro milhıes de clientes,nan,"SOCINAL S.A. CREDITO, FINANCIAMENTO E INVESTIM...",2 trimestre,15,2,4,5946,21,Banco/financeira
1,dee2867e-81dc-4cf6-b430-763fddef7160,2021,Top 10,2.77,"MIDWAY S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO",1 trimestre,50,91,30,10797493,171,Banco/financeira
2,57084e96-6c17-48ff-a445-53b07b046d8d,2017,Menos de quatro milhıes de clientes,nan,VOLVO (conglomerado),2 trimestre,4,0,1,10123,5,Conglomerado
3,c8cc112b-59a3-40bb-acb2-b902fb6adb2c,2019,Menos de quatro milhıes de clientes,nan,BANCOOB (conglomerado),4 trimestre,19,24,9,3558581,52,Conglomerado
4,6e5796cf-80e3-4128-9124-6350e21e13d7,2016,Menos de quatro milhıes de clientes,213.48,INTERMEDIUM (conglomerado),2 semestre,112,309,45,210784,466,Conglomerado
...,...,...,...,...,...,...,...,...,...,...,...,...
2483,04433211-6eee-437c-8437-19ab25539601,2016,Menos de quatro milhıes de clientes,nan,DEUTSCHE (conglomerado),2 semestre,1,0,0,4191,1,Conglomerado
2484,7502466f-7c53-4db9-92f8-99eab6559eec,2019,Menos de quatro milhıes de clientes,nan,SOCIETE GENERALE (conglomerado),4 trimestre,0,0,1,677,1,Conglomerado
2485,0c699987-d8aa-421e-aa4a-9e141bd58604,2018,Menos de quatro milhıes de clientes,nan,RENDIMENTO (conglomerado),1 trimestre,3,2,7,17782,12,Conglomerado
2486,3c676514-9a19-4f7b-b085-fd6695c1ab9e,2019,Menos de quatro milhıes de clientes,nan,HONDA (conglomerado),3 trimestre,20,2,3,475800,25,Conglomerado


In [73]:
# Excluindo a coluna 'chave' que não será utilizada nas análises

bc_reclame2.drop(columns = ['chave'],inplace = True)

In [74]:
# Verificando a exclusão da coluna

bc_reclame2.dtypes

ano                        int64
categoria                 object
indice_rec                object
instituicao_financeira    object
periodo                   object
qtde_rec_nao_reg           int64
qtde_rec_reg_nao_proc      int64
qtde_rec_reg_proc          int64
qtde_total_clientes        int64
qtde_total_rec             int64
tipo                      object
dtype: object

## Criando o DF de PySpark a partir do DF Pandas

In [ ]:
# Criando a SparkSession

spark = (
    SparkSession
      .builder
      .master('local')
      .appName('bc_reclame')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

In [ ]:
'''
# Criando o StructType

esquema = (
    StructType()
    .add('ano', IntegerType(), True)
    .add('categoria', StringType(), True)
    .add('indice_rec', StringType(), True)
    .add('instituicao_financeira', StringType(), True)
    .add('periodo', StringType(), True)
    .add('qtde_rec_nao_reg', IntegerType(), True)
    .add('qtde_rec_reg_nao_proc', IntegerType(), True)
    .add('qtde_rec_reg_proc', IntegerType(), True)
    .add('qtde_total_clientes', IntegerType(), True)
    .add('qtde_total_rec', IntegerType(), True)
)
'''

In [76]:
# Criando o DF de Spark a partir do DF de Pandas

bc_reclame_spark = spark.createDataFrame(bc_reclame2)

In [77]:
# Verificando a criação do DF

bc_reclame_spark.show()

+----+--------------------+----------+----------------------+-----------+----------------+---------------------+-----------------+-------------------+--------------+----------------+
| ano|           categoria|indice_rec|instituicao_financeira|    periodo|qtde_rec_nao_reg|qtde_rec_reg_nao_proc|qtde_rec_reg_proc|qtde_total_clientes|qtde_total_rec|            tipo|
+----+--------------------+----------+----------------------+-----------+----------------+---------------------+-----------------+-------------------+--------------+----------------+
|2019|Menos de quatro m...|       nan|  SOCINAL S.A. CRED...|2 trimestre|              15|                    2|                4|               5946|            21|Banco/financeira|
|2021|              Top 10|      2.77|  MIDWAY S.A. CREDI...|1 trimestre|              50|                   91|               30|           10797493|           171|Banco/financeira|
|2017|Menos de quatro m...|       nan|  VOLVO (conglomerado)|2 trimestre|            

In [78]:
# Verificando os tipos de dados em cada coluna

bc_reclame_spark.printSchema()

root
 |-- ano: long (nullable = true)
 |-- categoria: string (nullable = true)
 |-- indice_rec: string (nullable = true)
 |-- instituicao_financeira: string (nullable = true)
 |-- periodo: string (nullable = true)
 |-- qtde_rec_nao_reg: long (nullable = true)
 |-- qtde_rec_reg_nao_proc: long (nullable = true)
 |-- qtde_rec_reg_proc: long (nullable = true)
 |-- qtde_total_clientes: long (nullable = true)
 |-- qtde_total_rec: long (nullable = true)
 |-- tipo: string (nullable = true)



## Transformações com PySpark

In [79]:
# Ordenar por ano, período e instituição financeira (ordem alfabética)

bc_reclame_spark = bc_reclame_spark.orderBy('ano', 'periodo', 'instituicao_financeira')

In [80]:
# Conferindo o sucesso da operação

bc_reclame_spark.show()

+----+--------------------+----------+----------------------+----------+----------------+---------------------+-----------------+-------------------+--------------+----------------+
| ano|           categoria|indice_rec|instituicao_financeira|   periodo|qtde_rec_nao_reg|qtde_rec_reg_nao_proc|qtde_rec_reg_proc|qtde_total_clientes|qtde_total_rec|            tipo|
+----+--------------------+----------+----------------------+----------+----------------+---------------------+-----------------+-------------------+--------------+----------------+
|2014|Menos de dois mil...|       nan|  ABC-BRASIL (congl...|2 semestre|               5|                    0|                1|              11953|             6|    Conglomerado|
|2014|Menos de dois mil...|       nan|  AGIPLAN FINANCEIR...|2 semestre|              97|                   46|               11|              99272|           154|Banco/financeira|
|2014|Menos de dois mil...|       nan|  AGORACRED S.A. SO...|2 semestre|               1| 

In [81]:
# Reordernando colunas

ordem = ['ano', 'periodo', 'instituicao_financeira', 'tipo', 'categoria', 'indice_rec', 'qtde_rec_reg_proc', 
         'qtde_rec_reg_nao_proc', 'qtde_rec_nao_reg', 'qtde_total_rec', 'qtde_total_clientes']

bc_reclame_spark = bc_reclame_spark.select(ordem)

In [82]:
# Verificando o sucesso da operação

bc_reclame_spark.show(1)

+----+----------+----------------------+------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
| ano|   periodo|instituicao_financeira|        tipo|           categoria|indice_rec|qtde_rec_reg_proc|qtde_rec_reg_nao_proc|qtde_rec_nao_reg|qtde_total_rec|qtde_total_clientes|
+----+----------+----------------------+------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
|2014|2 semestre|  ABC-BRASIL (congl...|Conglomerado|Menos de dois mil...|       nan|                1|                    0|               5|             6|              11953|
+----+----------+----------------------+------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
only showing top 1 row



In [83]:
# Alterando o tipo de dados na coluna 'indice_rec'

bc_reclame_spark = bc_reclame_spark.withColumn('indice_rec', bc_reclame_spark['indice_rec'].cast(DoubleType()))

In [84]:
# Verificando o sucesso da operação

bc_reclame_spark.printSchema()

root
 |-- ano: long (nullable = true)
 |-- periodo: string (nullable = true)
 |-- instituicao_financeira: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- indice_rec: double (nullable = true)
 |-- qtde_rec_reg_proc: long (nullable = true)
 |-- qtde_rec_reg_nao_proc: long (nullable = true)
 |-- qtde_rec_nao_reg: long (nullable = true)
 |-- qtde_total_rec: long (nullable = true)
 |-- qtde_total_clientes: long (nullable = true)



In [85]:
# Salvando uma cópia de backup do DF PySpark

bc_reclame_spark_backup = bc_reclame_spark.alias('bc_reclame_spark_backup')

In [86]:
bc_reclame_spark_backup.show()

+----+----------+----------------------+----------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
| ano|   periodo|instituicao_financeira|            tipo|           categoria|indice_rec|qtde_rec_reg_proc|qtde_rec_reg_nao_proc|qtde_rec_nao_reg|qtde_total_rec|qtde_total_clientes|
+----+----------+----------------------+----------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
|2014|2 semestre|  ABC-BRASIL (congl...|    Conglomerado|Menos de dois mil...|       NaN|                1|                    0|               5|             6|              11953|
|2014|2 semestre|  AGIPLAN FINANCEIR...|Banco/financeira|Menos de dois mil...|       NaN|               11|                   46|              97|           154|              99272|
|2014|2 semestre|  AGORACRED S.A. SO...|Banco/financeira|Menos de dois mil...|       NaN| 

In [56]:
# Calculando o índice de reclamações para todas as linhas

bc_reclame_spark = (
    bc_reclame_spark.withColumn('indice_rec', 
    F.format_number(F.col('qtde_rec_reg_proc') * 1000000 / F.col('qtde_total_clientes'), 2))
)

In [59]:
bc_reclame_spark.show()

+----+----------+----------------------+----------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
| ano|   periodo|instituicao_financeira|            tipo|           categoria|indice_rec|qtde_rec_reg_proc|qtde_rec_reg_nao_proc|qtde_rec_nao_reg|qtde_total_rec|qtde_total_clientes|
+----+----------+----------------------+----------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
|2014|2 semestre|  ABC-BRASIL (congl...|    Conglomerado|Menos de dois mil...|     83.66|                1|                    0|               5|             6|              11953|
|2014|2 semestre|  AGIPLAN FINANCEIR...|Banco/financeira|Menos de dois mil...|    110.81|               11|                   46|              97|           154|              99272|
|2014|2 semestre|  AGORACRED S.A. SO...|Banco/financeira|Menos de dois mil...|     61.29| 

# Salvando o DF PySpark em parquet no Data Lake

In [60]:
# Salvando o DF em parquet no Data Lake

(
bc_reclame_spark.write.format("parquet")
.option("header", "true")
.option("inferschema", "true")
.save("gs://deloitte_g10/saída/Pyspark/Bc_Reclame/Reclame_norm_spark")
)

In [87]:
# Salvando o DF sem alteração na coluna 'indice_rec' no Data Lake

(
bc_reclame_spark_backup.write.format("parquet")
.option("header", "true")
.option("inferschema", "true")
.save("gs://deloitte_g10/saída/Pyspark/Bc_Reclame/Reclame_norm_spark_indiceNaoAlter")
)

# Consultas com SparkSQL

In [23]:
bc_reclame_spark.printSchema()

root
 |-- ano: long (nullable = true)
 |-- periodo: string (nullable = true)
 |-- instituicao_financeira: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- indice_rec: double (nullable = true)
 |-- qtde_rec_reg_proc: long (nullable = true)
 |-- qtde_rec_reg_nao_proc: long (nullable = true)
 |-- qtde_rec_nao_reg: long (nullable = true)
 |-- qtde_total_rec: long (nullable = true)
 |-- qtde_total_clientes: long (nullable = true)



In [26]:
# Criando a tabela temporária para consultas SQL

bc_reclame_spark.createOrReplaceTempView('reclame')

In [27]:
# Consulta teste

spark.sql('''
SELECT *
FROM reclame

''').show()

+----+----------+----------------------+----------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
| ano|   periodo|instituicao_financeira|            tipo|           categoria|indice_rec|qtde_rec_reg_proc|qtde_rec_reg_nao_proc|qtde_rec_nao_reg|qtde_total_rec|qtde_total_clientes|
+----+----------+----------------------+----------------+--------------------+----------+-----------------+---------------------+----------------+--------------+-------------------+
|2014|2 semestre|  ABC-BRASIL (congl...|    Conglomerado|Menos de dois mil...|       NaN|                1|                    0|               5|             6|              11953|
|2014|2 semestre|  AGIPLAN FINANCEIR...|Banco/financeira|Menos de dois mil...|       NaN|               11|                   46|              97|           154|              99272|
|2014|2 semestre|  AGORACRED S.A. SO...|Banco/financeira|Menos de dois mil...|       NaN| 

In [90]:
# Conferindo o número de linhas

spark.sql('''
SELECT COUNT(*)
FROM reclame
''').show()

+--------+
|count(1)|
+--------+
|    2488|
+--------+



In [30]:
# Conferindo tipos de dados da tabela temporária

spark.sql('''
DESCRIBE reclame

''').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                 ano|   bigint|   null|
|             periodo|   string|   null|
|instituicao_finan...|   string|   null|
|                tipo|   string|   null|
|           categoria|   string|   null|
|          indice_rec|   double|   null|
|   qtde_rec_reg_proc|   bigint|   null|
|qtde_rec_reg_nao_...|   bigint|   null|
|    qtde_rec_nao_reg|   bigint|   null|
|      qtde_total_rec|   bigint|   null|
| qtde_total_clientes|   bigint|   null|
+--------------------+---------+-------+



In [31]:
# Soma das reclamações procedentes agrupada por bancos (ranking 10)

spark.sql('''
SELECT 
    instituicao_financeira,
    SUM(qtde_rec_reg_proc) AS soma_rec_proc
FROM 
    reclame
GROUP BY
    instituicao_financeira
ORDER BY 
    soma_rec_proc DESC
'''
).show(10)

+----------------------+-------------+
|instituicao_financeira|soma_rec_proc|
+----------------------+-------------+
|  CAIXA ECONOMICA F...|        77166|
|  BRADESCO (conglom...|        64190|
|   ITAU (conglomerado)|        41879|
|     BB (conglomerado)|        35921|
|  SANTANDER (conglo...|        32869|
|  C6 BANK (conglome...|        19887|
|    PAN (conglomerado)|        16592|
|  BNP PARIBAS (cong...|        13379|
|    BMG (conglomerado)|        12770|
|  SAFRA (conglomerado)|        11771|
+----------------------+-------------+
only showing top 10 rows



In [39]:
# Procurar por instituições que contenham determinadas palavras

spark.sql('''
SELECT instituicao_financeira
FROM reclame
WHERE instituicao_financeira 
LIKE "%CAIXA%"
GROUP BY instituicao_financeira
''').show(30)

+----------------------+
|instituicao_financeira|
+----------------------+
|  CAIXA ECONOMICA F...|
|  CAIXA GERAL (cong...|
+----------------------+



In [41]:
# Ranking de número de clientes no período mais recente

spark.sql('''
SELECT 
    instituicao_financeira,
    qtde_total_clientes
FROM 
    reclame
WHERE
    ano = 2021 AND
    periodo = '3 trimestre'
ORDER BY 
    qtde_total_clientes DESC
''').show(30)

+----------------------+-------------------+
|instituicao_financeira|qtde_total_clientes|
+----------------------+-------------------+
|  CAIXA ECONOMICA F...|          145914729|
|  BRADESCO (conglom...|          100264792|
|   ITAU (conglomerado)|           87568455|
|     BB (conglomerado)|           70259170|
|  SANTANDER (conglo...|           53358153|
|  NUBANK (conglomer...|           22267718|
|  VOTORANTIM (congl...|           18414652|
|  BTG PACTUAL/BANCO...|           15250442|
|  INTER (conglomerado)|           13323223|
|  C6 BANK (conglome...|           11046743|
|  MIDWAY S.A. CREDI...|           10572319|
|  BANCO DO NORDESTE...|            8844660|
|    BMG (conglomerado)|            8140173|
|        BANCO CSF S.A.|            7425297|
|  PEFISA S.A. - CRE...|            6798337|
|  SICREDI (conglome...|            6101332|
|  ORIGINAL (conglom...|            5634279|
|   OMNI (conglomerado)|            5281196|
|  BANCO SICOOB (con...|            5047970|
|  BANRISU

In [42]:
# Ranking de número de clientes no período mais antigo

spark.sql('''
SELECT 
    instituicao_financeira,
    qtde_total_clientes
FROM 
    reclame
WHERE
    ano = 2014 AND
    periodo = '2 semestre'
ORDER BY 
    qtde_total_clientes DESC
''').show(30)

+----------------------+-------------------+
|instituicao_financeira|qtde_total_clientes|
+----------------------+-------------------+
|  BRADESCO (conglom...|           74105981|
|  CAIXA ECONOMICA F...|           73691832|
|   ITAU (conglomerado)|           56877696|
|     BB (conglomerado)|           55124936|
|  SANTANDER (conglo...|           31102625|
|   HSBC (conglomerado)|           10161845|
|  VOTORANTIM (congl...|            4434210|
|  BANCO DO NORDESTE...|            4344516|
|  BANRISUL (conglom...|            3834547|
|    BMG (conglomerado)|            2272314|
|  MERCANTIL DO BRAS...|            2102624|
|  PANAMERICANO (con...|            1847598|
|  SICREDI (conglome...|            1819046|
|        BANCO CSF S.A.|            1414441|
|  BNP PARIBAS (cong...|            1167917|
|  BANESTES (conglom...|            1158114|
|  BANCOOB (conglome...|            1156296|
|  CITIBANK (conglom...|            1056849|
|  PERNAMBUCANAS (co...|            1014196|
|  PORTO S

In [43]:
# Ranking de número de clientes imediatamente anterior à pandemia

spark.sql('''
SELECT 
    instituicao_financeira,
    qtde_total_clientes
FROM 
    reclame
WHERE
    ano = 2019 AND
    periodo = '4 trimestre'
ORDER BY 
    qtde_total_clientes DESC
''').show(30)

+----------------------+-------------------+
|instituicao_financeira|qtde_total_clientes|
+----------------------+-------------------+
|  BRADESCO (conglom...|           98653100|
|  CAIXA ECONOMICA F...|           95191144|
|   ITAU (conglomerado)|           81684646|
|     BB (conglomerado)|           66464386|
|  SANTANDER (conglo...|           47281933|
|  MIDWAY S.A. CREDI...|           11392747|
|  BANCO DO NORDESTE...|            7836766|
|  VOTORANTIM (congl...|            7364173|
|        BANCO CSF S.A.|            7235478|
|  BANRISUL (conglom...|            4935064|
|  REALIZE CREDITO, ...|            4862925|
|    BMG (conglomerado)|            4747780|
|   OMNI (conglomerado)|            4623207|
|  SICREDI (conglome...|            4564825|
|    PAN (conglomerado)|            4459480|
|  INTER (conglomerado)|            3957845|
|  BANCOOB (conglome...|            3558581|
|  PERNAMBUCANAS FIN...|            3301184|
|  BNP PARIBAS (cong...|            3162634|
|  MERCANT

In [44]:
# Ranking de número de clientes ao fim de 2020, um ano depois, durante a pandemia

spark.sql('''
SELECT 
    instituicao_financeira,
    qtde_total_clientes
FROM 
    reclame
WHERE
    ano = 2020 AND
    periodo = '4 trimestre'
ORDER BY 
    qtde_total_clientes DESC
''').show(30)

+----------------------+-------------------+
|instituicao_financeira|qtde_total_clientes|
+----------------------+-------------------+
|  CAIXA ECONOMICA F...|          143971402|
|  BRADESCO (conglom...|           99362077|
|   ITAU (conglomerado)|           83674760|
|     BB (conglomerado)|           69012481|
|  SANTANDER (conglo...|           50947844|
|  VOTORANTIM (congl...|           13112975|
|  MIDWAY S.A. CREDI...|           10768835|
|  NUBANK (conglomer...|           10124543|
|  BANCO DO NORDESTE...|            8360626|
|  INTER (conglomerado)|            8069837|
|        BANCO CSF S.A.|            7498821|
|    PAN (conglomerado)|            7081960|
|    BMG (conglomerado)|            6051361|
|  PEFISA S.A. - CRE...|            5897857|
|  SICREDI (conglome...|            5384094|
|  BANRISUL (conglom...|            5127105|
|  REALIZE CREDITO, ...|            4891419|
|   OMNI (conglomerado)|            4606413|
|  C6 BANK (conglome...|            4384238|
|  BANCOOB